# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    size='Humidity', 
    color='City', 
    hover_cols=['City', 'Country', 'Humidity'],
    title='City Map with Humidity',
)

# Display the map
map_plot

:Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 20) & 
    (city_data_df['Humidity'] < 60)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
12,trairi,BR,-3.2778,-39.2689,51,
22,guiratinga,BR,-16.3494,-53.7583,14,
29,saint-louis,SN,16.3333,-15.0000,24,
34,naberera,TZ,-4.2000,36.9333,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# https://api.geoapify.com/v2/places?categories=catering.restaurant,catering.cafe&filter=rect:11.573106549898483,48.13898913611139,11.57704581350751,48.13666585409989&limit=20&apiKey=YOUR_API_KEY
# Set parameters to search for a hotel 

BASE_URL = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
RADIUS = 50000
params = {
    'apiKey': geoapify_key,
    'limit': 1,  # Get the first hotel
    'categories': 'accommodation',
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{RADIUS}'
    params["bias"] = f'proximity:{longitude},{latitude}'


    # Make and API request using the params dictionary
    response = requests.get(BASE_URL, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: فندق إنتركونتيننتال
trairi - nearest hotel: Pousada Villa Aurora
guiratinga - nearest hotel: estrela contêiner
saint-louis - nearest hotel: La Cour du Fleuve
naberera - nearest hotel: No hotel found
cayenne - nearest hotel: Central Hôtel
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: فندق القصر
rafaela - nearest hotel: Parra Hotel & Suites
itarema - nearest hotel: Pousada Oásis
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
sao miguel do araguaia - nearest hotel: Hotel São Miguel
aripuana - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
chunhuhub - nearest hotel: Kiich pam kaax ( Selva bonita)
ouallam - nearest hotel: No hotel found
shakawe - nearest hotel: EdenWood guesthouse
upington - nearest hotel: Aardwolf Backpackers
gwanda - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,فندق إنتركونتيننتال
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora
22,guiratinga,BR,-16.3494,-53.7583,14,estrela contêiner
29,saint-louis,SN,16.3333,-15.0000,24,La Cour du Fleuve
34,naberera,TZ,-4.2000,36.9333,54,No hotel found
...,...,...,...,...,...,...
536,nouadhibou,MR,20.9310,-17.0347,27,الحانة شنقيط
542,bilma,NE,18.6853,12.9164,18,No hotel found
545,kalabo,ZM,-14.9700,22.6814,14,Golden Lodge Luxury Acomodation
550,pali,IN,25.7667,73.3333,26,Hotel Pawan International


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    size='Humidity', 
    color='City', 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    title='City Map with Hotels and Humidity'
)

# Display the map
hotel_map

:Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)